In [63]:
import os
import warnings
from datetime import datetime 
import pandas as pd
import numpy as np
from numpy.random import seed
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import ColumnTransformer
import seaborn as sns
import matplotlib.pyplot as plt
plt.rcParams['figure.facecolor'] = 'white'
warnings.simplefilter('ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'


In [64]:
def create_dir(path):
        isExist = os.path.exists(path)
        if not isExist:
            os.makedirs(path, exist_ok = False)
            print("New directory is created")



In [69]:
class DataFormatting():
  
    def __init__(self):
       
        self.df_data = None
        self.df_datetime = None

    def dataset(df):

        # converting time colum from object type to datetime format
        df['time'] = pd.to_datetime(df['time'])
        # splitting the dataframe in to X and y 
        df_data = df[['open','high','low','close','tick_volume']]
        df_datetime =df[['time']]

        return df_data, df_datetime


data = pd.read_csv('../data/gold_mt5.csv',index_col=[0]) 

data_init = DataFormatting()
df_data, df_datetime = DataFormatting.dataset(data)
print(df_data.head())


     open    high    low  close  tick_volume
0  310.30  314.40  310.0  313.5          561
1  312.60  315.20  311.9  314.3          491
2  313.40  314.20  311.8  312.9          431
3  312.25  313.65  308.9  309.9          716
4  309.39  310.90  306.3  308.2          802


In [74]:
future_days = 5

forecast_date = pd.date_range(list(df_datetime.time)[-1], periods = future_days, freq = '1d').tolist()


In [75]:
forecast_date

[Timestamp('2022-07-28 00:00:00', freq='D'),
 Timestamp('2022-07-29 00:00:00', freq='D'),
 Timestamp('2022-07-30 00:00:00', freq='D'),
 Timestamp('2022-07-31 00:00:00', freq='D'),
 Timestamp('2022-08-01 00:00:00', freq='D')]

In [44]:
def train_test_split(data, train_split=0.7):

    """ This function will split the dataframe into training and testing set.
    Inputs: data: Pandas DatFrame
            train_split: default is set to 0.9. Its a ratio to split the trining and testing datset.
    """
    split = int(train_split*len(data)) # for training
    split_test = int(0.90*len(data))
    X_train = data.iloc[:split,:]
    X_val = data.iloc[split:split_test,:]
    X_test = data.iloc[split_test:,:]

    return X_train, X_val, X_test

X_train, X_val, X_test = train_test_split(df_data, train_split=0.7)

print(X_train.tail(), X_val.head(), X_val.tail(),X_test.head(), X_test.tail())


         open     high      low    close  tick_volume
4499  1291.91  1293.77  1279.67  1284.54        69923
4500  1283.21  1285.92  1251.96  1257.16        99537
4501  1257.03  1284.76  1256.42  1283.33        82008
4502  1282.10  1283.40  1266.41  1273.99        78714
4503  1273.45  1285.76  1255.48  1260.31        84037          open     high      low    close  tick_volume
4504  1260.34  1272.52  1256.67  1269.39        81582
4505  1268.51  1274.05  1256.35  1264.64        76837
4506  1264.50  1268.74  1228.29  1233.65        86849
4507  1235.94  1243.64  1234.30  1239.14        53069
4508  1238.73  1245.85  1230.98  1233.38        60960          open     high      low    close  tick_volume
5786  1562.49  1575.83  1556.56  1571.43       115510
5787  1581.21  1588.51  1575.90  1581.86       161267
5788  1581.58  1583.07  1565.57  1567.88       126826
5789  1566.89  1578.04  1563.40  1576.76       111628
5790  1576.86  1585.99  1572.30  1573.89       163993          open     high      

In [ ]:
# class Normalize():
    
#     """ class Normalize uses standard scaler method to normalize the dataset"""
#     def __init__(self):

#         self.data_fit_transformed = None
#         self.data_inverse_transformed = None

#     def fit_transform(self, data):

#         # initialize StandartScaler()
#         scaler = StandardScaler()
#         # fit the method on the dataset
#         scaler = scaler.fit(data)
#         # transform the dataset
#         data_fit_transformed = scaler.transform(data)

#         return data_fit_transformed

#     def transform(self,data):
    
#         # initialize StandardScaler()
#         scaler = StandardScaler()

#         # transform the dataset
#         data_fit_transformed = scaler.transform(data)

#         return data_fit_transformed

#     def inverse_transform(self, data):

#         # initialize StandartScaler()
#         scaler = StandardScaler()
#         # inverse transform the dataset
#         data_inverse_transformed = scaler.inverse_transform(data)
        
#         return data_inverse_transformed

In [27]:
class Normalize():

    """ class Normalize uses standard scaler method to normalize the dataset"""
    def __init__(self):

        self.data_fit_transformed = None
        self.data_inverse_transformed = None

    def fit_transform(self, data_train, data_val, data_test):

        # initialize StandartScaler()
        scaler = StandardScaler()
        # define transformer
        transformer = [('standard_scaler', StandardScaler(),['open','high','low','close','tick_volume'])]
        # define column transformer
        column_transformer = ColumnTransformer(transformers = transformer)
        # fit and transform training data
        data_fit_transformed = column_transformer.fit_transform(data_train)
        # # fit the method on the dataset
        # scaler = scaler.fit(data)
        # # transform the dataset
        # data_fit_transformed = scaler.transform(data)
        val_transformed = column_transformer.transform(data_val)
        test_transformed = column_transformer.transform(data_test)
        return data_fit_transformed, val_transformed, test_transformed

    def transform(self,data):

        # initialize StandartScaler()
        scaler = StandardScaler()

        # transform the dataset
        data_fit_transformed = scaler.transform(data)

        return data_fit_transformed

    def inverse_transform(self, data):

        # initialize StandartScaler()
        scaler = StandardScaler()
        # inverse transform the dataset
        data_inverse_transformed = scaler.inverse_transform(data)
        
        return data_inverse_transformed

# normalize
scaler_init = Normalize()
data_fit_transformed, val_transformed, test_transformed = scaler_init.fit_transform(X_train, X_val, X_test)
print(data_fit_transformed[0:5], val_transformed[0:5], test_transformed[0:5])


[[-0.96366767 -0.9621959  -0.95901081 -0.95840204 -0.71730974]
 [-0.95902447 -0.96059173 -0.95514351 -0.95678648 -0.72034248]
 [-0.95740944 -0.96259695 -0.95534705 -0.95961372 -0.72294197]
 [-0.95973104 -0.96369981 -0.96124978 -0.96567209 -0.71059439]
 [-0.96550476 -0.96921414 -0.96654188 -0.96910517 -0.70686845]] [[0.95425693 0.95903778 0.96786237 0.97197787 2.79291116]
 [0.97075039 0.96210576 0.96721104 0.96238544 2.58733485]
 [0.96265507 0.95145808 0.91009709 0.89980243 3.02110303]
 [0.90499863 0.90112726 0.92232997 0.91088926 1.55759034]
 [0.91063103 0.90555878 0.91557237 0.89925718 1.89946655]] [[1.58783179 1.59653477 1.60761594 1.61776022 6.2534818 ]
 [1.62606754 1.6029314  1.6050106  1.59251699 6.04604253]
 [1.59304025 1.57495869 1.56304019 1.54411058 5.00187084]
 [1.54424626 1.54034873 1.55984458 1.55200666 5.91138896]
 [1.55100919 1.55209927 1.57008275 1.57220124 4.83212417]]


In [29]:
def data_transformation(data, lags = 5):

    """ this function transforms dataframe to required input shape for the model.
    It required 2 input arguments:
    1. data: this will be the pandas dataframe
    2. lags: how many previous price points to be used to predict the next future value, in
    this case the default is set to 5 for 'XAUUSD' commodity"""

    # initialize lists to store the dataset
    X_data = []
    y_data = []
    
    for i in range(lags, len(data)):
        X_data.append(data[i-lags: i, 0: data.shape[1]])
        y_data.append(data[i,3:4]) # extracts close price with specific lag as price to be predicted.

    # convert the list to numpy array

    X_data = np.array(X_data)
    y_data = np.array(y_data)

    return X_data, y_data


X_data, y_data = data_transformation(data_fit_transformed, lags = 5)
X_val_data, y_val_data =  data_transformation(val_transformed, lags = 5)
X_test_data, y_test_data =  data_transformation(test_transformed, lags = 5)


In [30]:
print(X_data.shape[0],X_data.shape[1],X_data.shape[2],y_data.shape[1])


4499 5 5 1


In [31]:
class LSTM_model():
    

    def __init__(self,n_hidden_layers, units, dropout, train_data_X, train_data_y, epochs):

        self.n_hidden_layers = n_hidden_layers
        self.units = units
        self.dropout = dropout
        self.train_data_X = train_data_X
        self.train_data_y = train_data_y
        self.epochs = epochs

    def build_model(self):
        
        model = Sequential()
        # first lstm layer
        model.add(LSTM(self.units, activation='relu', input_shape=(self.train_data_X.shape[1], self.train_data_X.shape[2]), return_sequences=True))
        # building hidden layers
        for i in range(1, self.n_hidden_layers):
            # for the last layer as the return sequence is False
            if i == self.n_hidden_layers -1:
                model.add(LSTM(int(self.units/(2**i)),  activation='relu', return_sequences=False))
            else:
                model.add(LSTM(int(self.units/(2**i)),  activation='relu', return_sequences=True))
        # adding droupout layer
        model.add(Dropout(self.dropout))
        # final layer
        model.add(Dense(self.train_data_y.shape[1]))
        return model
        


In [32]:
if __name__ == '__main__':

    seed(42)
    tf.random.set_seed(42) 
    keras.backend.clear_session()

    n_hidden_layers = 3
    units = 128
    dropout = 0.2
    train_data_X = X_data 
    train_data_y = y_data
    epochs = 5

    # creating main folder
    today = datetime.now()
    today  = today.strftime('%Y_%m_%d')
    path = '../Model_Outputs/'+ today
    create_dir(path)
 
    # creating directory to save model and its output
    folder = 'model_lstm'+ str(units)
    path_main = path + '/'+ folder
    create_dir(path_main)

    # creating directory to save all the metric data
    folder = 'metrics'
    path_metrics = path_main +'/'+ folder
    create_dir(path_metrics)

    # creating folder to save model.h5 file
    folder = 'model'
    path_model = path_main +'/'+ folder
    create_dir(path_model)

    # creating folder to save model.h5 file
    folder = 'model_checkpoint'
    path_checkpoint = path_main +'/'+ folder
    create_dir(path_checkpoint)

    # initializing model
    model_init = LSTM_model(n_hidden_layers, units, dropout, train_data_X, train_data_y, epochs)

    # calling the model
    model = model_init.build_model()

    # metrics for evaluating the model
    metrics = [tf.keras.metrics.RootMeanSquaredError(), tf.keras.metrics.MeanAbsoluteError(), tf.keras.metrics.MeanAbsolutePercentageError()]

    # model compiler
    model.compile(optimizer=Adam(learning_rate = 0.0001), loss='mse', metrics = metrics)

    # setting the model file name
    model_name = 'lstm_'+ str(units)+'.h5'
    
    # setting the callback function
    cb = [
        tf.keras.callbacks.ModelCheckpoint(path_checkpoint),
        tf.keras.callbacks.CSVLogger(path_metrics+'/'+'data.csv'),
        tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=False)]

    # model fitting protocol
    history = model.fit(train_data_X,train_data_y, 
                        epochs = epochs, 
                        batch_size = 8, 
                        validation_data=(X_val_data, y_val_data), 
                        verbose = 1,
                        callbacks=[cb],
                        shuffle= False)

    # path to save model
    model.save(path_model+'/'+model_name)


Epoch 1/5
563/563 [==============================] - ETA: 0s - loss: 0.2691 - root_mean_squared_error: 0.5188 - mean_absolute_error: 0.3852 - mean_absolute_percentage_error: 58.5194INFO:tensorflow:Assets written to: ../Model_Outputs/2022_07_30/model_lstm128\model_checkpoint\assets


563/563 [==============================] - 23s 36ms/step - loss: 0.2691 - root_mean_squared_error: 0.5188 - mean_absolute_error: 0.3852 - mean_absolute_percentage_error: 58.5194 - val_loss: 4.6575 - val_root_mean_squared_error: 2.1581 - val_mean_absolute_error: 1.2597 - val_mean_absolute_percentage_error: 124.0693
Epoch 2/5
563/563 [==============================] - ETA: 0s - loss: 0.0544 - root_mean_squared_error: 0.2333 - mean_absolute_error: 0.1620 - mean_absolute_percentage_error: 31.9824INFO:tensorflow:Assets written to: ../Model_Outputs/2022_07_30/model_lstm128\model_checkpoint\assets


INFO:tensorflow:Assets written to: ../Model_Outputs/2022_07_30/model_lstm128\model_checkpoint\assets


563/563 [==============================] - 20s 36ms/step - loss: 0.0544 - root_mean_squared_error: 0.2333 - mean_absolute_error: 0.1620 - mean_absolute_percentage_error: 31.9824 - val_loss: 1.8314 - val_root_mean_squared_error: 1.3533 - val_mean_absolute_error: 0.8410 - val_mean_absolute_percentage_error: 83.0830
Epoch 3/5
563/563 [==============================] - ETA: 0s - loss: 0.0532 - root_mean_squared_error: 0.2307 - mean_absolute_error: 0.1573 - mean_absolute_percentage_error: 32.7881INFO:tensorflow:Assets written to: ../Model_Outputs/2022_07_30/model_lstm128\model_checkpoint\assets


INFO:tensorflow:Assets written to: ../Model_Outputs/2022_07_30/model_lstm128\model_checkpoint\assets


563/563 [==============================] - 26s 45ms/step - loss: 0.0532 - root_mean_squared_error: 0.2307 - mean_absolute_error: 0.1573 - mean_absolute_percentage_error: 32.7881 - val_loss: 0.5029 - val_root_mean_squared_error: 0.7092 - val_mean_absolute_error: 0.4800 - val_mean_absolute_percentage_error: 47.4891
Epoch 4/5
562/563 [============================>.] - ETA: 0s - loss: 0.0389 - root_mean_squared_error: 0.1971 - mean_absolute_error: 0.1385 - mean_absolute_percentage_error: 32.6711INFO:tensorflow:Assets written to: ../Model_Outputs/2022_07_30/model_lstm128\model_checkpoint\assets


INFO:tensorflow:Assets written to: ../Model_Outputs/2022_07_30/model_lstm128\model_checkpoint\assets


563/563 [==============================] - 24s 42ms/step - loss: 0.0389 - root_mean_squared_error: 0.1971 - mean_absolute_error: 0.1385 - mean_absolute_percentage_error: 32.6618 - val_loss: 0.0152 - val_root_mean_squared_error: 0.1233 - val_mean_absolute_error: 0.0900 - val_mean_absolute_percentage_error: 8.6515
Epoch 5/5
562/563 [============================>.] - ETA: 0s - loss: 0.0346 - root_mean_squared_error: 0.1860 - mean_absolute_error: 0.1320 - mean_absolute_percentage_error: 27.7087INFO:tensorflow:Assets written to: ../Model_Outputs/2022_07_30/model_lstm128\model_checkpoint\assets


INFO:tensorflow:Assets written to: ../Model_Outputs/2022_07_30/model_lstm128\model_checkpoint\assets


563/563 [==============================] - 25s 44ms/step - loss: 0.0346 - root_mean_squared_error: 0.1860 - mean_absolute_error: 0.1320 - mean_absolute_percentage_error: 27.7014 - val_loss: 0.0141 - val_root_mean_squared_error: 0.1186 - val_mean_absolute_error: 0.0619 - val_mean_absolute_percentage_error: 5.9191


In [60]:
train_loss, RMSE, MAE, MAPE = model.evaluate(train_data_X,train_data_y)
print('\n','Evaluation of Training dataset:','\n','train_loss:',round(train_loss,3),'\n','RMSE:',round(RMSE,3),'\n', 'MAE:',round(MAE,3),'\n','MAPE:',round(MAPE,3))
print('\n')
    

141/141 [==============================] - 1s 8ms/step - loss: 0.0373 - root_mean_squared_error: 0.1930 - mean_absolute_error: 0.1409 - mean_absolute_percentage_error: 32.2084

 Evaluation of Training dataset: 
 train_loss: 0.037 
 RMSE: 0.193 
 MAE: 0.141 
 MAPE: 32.208




In [62]:
val_loss, val_RMSE, val_MAE, val_MAPE = model.evaluate(X_val_data, y_val_data)
print('\n','Evaluation of Validation dataset:','\n','train_loss:',round(val_loss,3),'\n','val_RMSE:',round(val_RMSE,3),'\n', 'val_MAE:',round(val_MAE,3),'\n','MAPE:',round(MAPE,3))

41/41 [==============================] - 0s 8ms/step - loss: 0.0141 - root_mean_squared_error: 0.1186 - mean_absolute_error: 0.0619 - mean_absolute_percentage_error: 5.9192

 Evaluation of Validation dataset: 
 train_loss: 0.014 
 val_RMSE: 0.119 
 val_MAE: 0.062 
 MAPE: 32.208


In [ ]:


def metricplot(df, xlab, ylab_1,ylab_2, path):
    
    """
    This function plots metric curves and saves it
    to respective folder
    inputs: df : pandas dataframe 
            xlab: x-axis
            ylab_1 : yaxis_1
            ylab_2 : yaxis_2
            path: full path for saving the plot
            """
    plt.figure()
    sns.set_theme(style="darkgrid")
    sns.lineplot(x = df[xlab], y = df[ylab_1])
    sns.lineplot(x = df[xlab], y = df[ylab_2])
    plt.xlabel('Epochs',fontsize = 12)
    plt.ylabel(ylab_1,fontsize = 12)
    plt.xticks(fontsize = 12)
    plt.yticks(fontsize = 12)
    plt.legend([ylab_1,ylab_2], prop={"size":12})
    plt.savefig(path+'/'+ ylab_1)
    #plt.show()


In [ ]:
data_met = pd.read_csv('../Model_Outputs/model_lstm/metrics/data.csv')
data_met

In [ ]:
path = '../Model_Outputs/model_lstm/metrics'
df = pd.read_csv('../Model_Outputs/model_lstm/metrics/data.csv')

metricplot(df, 'epoch', 'loss','val_loss', path)
metricplot(df, 'epoch', 'mean_absolute_error','val_mean_absolute_error', path)
metricplot(df, 'epoch', 'mean_absolute_percentage_error','val_mean_absolute_percentage_error', path)
metricplot(df, 'epoch', 'root_mean_squared_error','val_root_mean_squared_error', path)


In [ ]:
from datetime import datetime
today = datetime.now()
today  = today.strftime('%Y%m%d')
path = '../Model_Outputs/'+ today